In [ ]:
# Logging 
# set up logging to file - see previous section for more details
def create_logging(OUTPUT_FOLDER):
    logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                        datefmt='%d-%m %H:%M:%S',
                        filename= os.path.join(OUTPUT_FOLDER, 'logger.log'),
                        filemode='w')

    # define a Handler which writes INFO messages or higher to the sys.stderr or sys.stdout
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    # set a format which is simpler for console use
    formatter = logging.Formatter(fmt='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                                  datefmt='%d-%m %H:%M')
    # tell the handler to use this format
    console.setFormatter(formatter)
    # add the handler to the root logger
    logging.getLogger().addHandler(console)
    return logging

In [ ]:
def read_df_print(path, format = "feather"): # TODO: can change to other formats too
  if format == "feather":
    df = pd.read_feather(path)
  print(df.shape)
  display(df.head(3))
  return df

In [ ]:
# Gets a feature as a sequence in lists (with pd.Series)
def get_user_sequence(feature):
  user_seq = train.groupby("user_id")[feature].apply(list)
  return user_seq

In [ ]:
# Exercises
def return_E():
  E_lists = get_user_sequence("content_id")
  return E_lists

In [ ]:
# Results r
def return_r():
  # Add results
  r_lists = get_user_sequence("answered_correctly") # All results (r)
  return r_lists

In [ ]:
# Results r
def return_r(add_start_token):
  # Add results
  r_lists = get_user_sequence("answered_correctly") # All results (r)

  # Add start token to r_list
  if add_start_token:
    r_lists = r_lists.apply(lambda x: [N_response+1] + x)
  return r_lists

In [ ]:
# Elapsed time - categorical
def return_etc():
    train["etc"] = train["prior_question_elapsed_time"]/1000.0
    
    # Add start token
    train["etc"] = train["etc"].fillna(301)
    train["etc"] = np.round(train["etc"]).astype(np.int32) # nearest integer

    etc_lists = get_user_sequence("etc") 
    del train["etc"]
    return etc_lists

In [ ]:
# Lag time - categorical
def return_ltc():
    train["ltc"] = train.groupby("user_id")["timestamp"].shift()
    
    # Lag in minutes
    train["ltc"] = ((train["timestamp"] - train["ltc"])/(1000.0 * 60))
    
    # Cap lag time to 1440 minutes
    train["ltc"] = np.clip(train["ltc"], 0, 1439)
    train["ltc"] = train["ltc"].fillna(1440)
    train["ltc"] = np.round(train["ltc"]).astype(np.int32) # nearest integer

    ltc_lists = get_user_sequence("ltc") 
    del train["ltc"]
    return ltc_lists

In [ ]:
# Attempt number
def return_attempt(max_attempts=8):
    train["attempt"] = train.groupby(["user_id", "content_id"]).cumcount()
    train["attempt"] = np.clip(train["attempt"], 0, max_attempts-1).astype(np.int16)
    at_lists = get_user_sequence("attempt")
    del train["attempt"]
    return at_lists

In [ ]:
# Content difficulty
def return_ca():
    ca = train.groupby("content_id")["answered_correctly"].transform('mean')
    train["ca"] = np.round(ca*100)
    train["ca"] = np.clip(train["ca"], 0, 100).astype(np.int32)
    ca = get_user_sequence("ca")
    del train["ca"]
    return ca

In [ ]:
# Part
def return_p():
    part_dict = dict(zip(questions.question_id, questions.part))
    train["part"]= train["content_id"].map(part_dict).fillna(0).astype("int8")
    p_lists = get_user_sequence("part") # All parts (p) of the exercises
    del train["part"]
    return p_lists

In [ ]:
# Prior question had explanation
def return_ex():
    train["ex"] = train["prior_question_had_explanation"] # *1 # => binary 0/1
    train["ex"] = train["ex"].fillna(3) # Start token
    ex_lists = get_user_sequence("ex")
    del train["ex"]
    return ex_lists

In [ ]:
# Average result
def return_r_avg():
    cums = train.groupby("user_id")["answered_correctly"].cumsum()
    cumc = train.groupby("user_id")["answered_correctly"].cumcount() + 1
    train["ra"] = round(100*cums/cumc)
    train["ra"] = np.clip(train["ra"], 0, 100).astype(np.int32)
    ra = get_user_sequence("ra")
    del train["ra"]
    return ra

In [ ]:
# Elapsed time - grouped
def return_etg(bins):
    train["etg"] = train["prior_question_elapsed_time"]/1000.0
    train["etg"] = np.clip(train["etg"], 0, 300)
    train["etg"] = np.round(train["etg"]) #.astype(np.int32)
    
    N_labels = len(bins)-1
    train["etg"] = pd.cut(train["etg"], bins, labels = range(N_labels))
    train["etg"] = np.array(train["etg"]) # Remove category 
    train["etg"] = train["etg"].fillna(N_labels+1).astype(np.int32)

    etg_lists = get_user_sequence("etg") 
    del train["etg"]
    return etg_lists

In [ ]:
# Lag time dense
def return_lt():
    train["lt"] = train.groupby("user_id")["timestamp"].shift()
    train["lt"] = train["timestamp"] - train["lt"]
    assert((train["lt"]<0).sum()==0) # There should be no negative time differences

    quantile_transformer = preprocessing.QuantileTransformer(random_state=0)
    train["lt"] = quantile_transformer.fit_transform(train["lt"].values.reshape(-1, 1))
    # train["lt"] = train["lt"].fillna(0.0) # Fill NA with 0.0? 0.5?
    train["lt"] = train["lt"].fillna(0.5)
        
    lt_lists = get_user_sequence("lt") 
    del train["lt"]
    return lt_lists, quantile_transformer

In [ ]:
# Elapsed time dense
def return_et():
    train["et"] = train["prior_question_elapsed_time"].fillna(0) # Replace "start" with zero

    quantile_transformer = preprocessing.QuantileTransformer(random_state=0)
    qt_transform_train = quantile_transformer.fit_transform(train.loc[train["user_id"].isin(train_users), "et"].values.reshape(-1, 1))
    qt_transform_val = quantile_transformer.transform(train.loc[train["user_id"].isin(val_users), "et"].values.reshape(-1, 1))
    
    train.loc[train["user_id"].isin(train_users), "et"] = qt_transform_train
    train.loc[train["user_id"].isin(val_users), "et"] = qt_transform_val

    et_lists = get_user_sequence("et") # Elapsed  times
    del train["et"]
    return et_lists, quantile_transformer

# Older code

In [ ]:
# # Elapsed time - grouped
# def return_etg(N_groups):
#     train["etg"] = train["prior_question_elapsed_time"]/1000.0
#     train["etg"] = np.clip(train["etg"], 0, 300)
#     train["etg"] = np.round(train["etg"]) #.astype(np.int32)
    
#     train["etg"] = pd.qcut(train["etg"], N_groups, labels=range(N_groups))
#     train["etg"] = np.array(train["etg"]) # Start token
#     train["etg"] = train["etg"].fillna(N_groups).astype(np.int32)
#     etg_lists = get_user_sequence("etg") 
#     del train["etg"]
#     return etg_lists

In [ ]:
# Seperately train/val
# qt_transform_train = quantile_transformer.fit_transform(train.loc[train["user_id"].isin(train_users), "lt"].values.reshape(-1, 1))
# qt_transform_val = quantile_transformer.transform(train.loc[train["user_id"].isin(val_users), "lt"].values.reshape(-1, 1))

# train.loc[train["user_id"].isin(train_users), "lt"] = qt_transform_train
# train.loc[train["user_id"].isin(val_users), "lt"] = qt_transform_val 